In [95]:
from gtm import *

# Sample Copulas Package
import pyvinecopulib as pv

# Other Stuff
import numpy as np
import torch
from torch.utils.data import DataLoader
from demos.dataset_helpers import Generic_Dataset

In [96]:
import torch

N = 100  # Number of samples
D = 5    # Dimensionality of Gaussian
K = 3    # Number of categories

# 1. Gaussian observations (mean=0, std=1)
y_train = torch.randn(N, D)  # Shape: (N, D)
y_val = torch.randn(N, D)  # Shape: (N, D)

# 2. Categorical identifier variable
x_train = torch.randint(0, K, (N,))  # N integers in {0, 1, ..., K-1}
x_val = torch.randint(0, K, (N,))

In [97]:
from gtm.gtm_layers.transformation_layer import Transformation

t_layer = Transformation(degree=[10] * D,number_variables=D,spline_range=[[-10]*D,[10]*D])

In [98]:
from gtm.gtm_splines.bernstein_prediction_vectorized import binomial_coeffs
device = "cpu"
max_degree = 10
n = max_degree + 1
binom_n = binomial_coeffs(n, device=device)
binom_n1 = binomial_coeffs(n - 1, device=device)
binom_n2 = binomial_coeffs(n - 2, device=device)

from gtm.gtm_layers.transformation_layer import Transformation
# define some params
params = torch.nn.ParameterList(t_layer.compute_starting_values(categories=3))

padded_params = torch.vstack(
            [
                torch.nn.functional.pad(
                    p, (0, 10 + 3 - 1 - p.size(0))
                )
                for p in params
            ]
        ).T

# define restriction for params
from gtm.gtm_splines.bernstein_basis import (
    compute_multivariate_bernstein_basis, restrict_parameters)

padded_params_restricted = restrict_parameters(padded_params,covariate=False, degree=10, monotonically_increasing=True)

# import get basis
from gtm.gtm_splines.bernstein_prediction_vectorized import bernstein_basis_batched
from gtm.gtm_splines.splines_utils import ReLULeR

reluler = ReLULeR(torch.FloatTensor([-10,10]))
input_a_clone = reluler.forward(y_train)

n = padded_params_restricted.shape[0] -1 #params_a.shape[0] - 1

# get into the 0,1 range
span_data = 10 - -10 #spline_range[1] - spline_range[0]
input_a_clone = input_a_clone / span_data + 0.5


basis = bernstein_basis_batched(input_a_clone, n, binom=binom_n)  # (N, D, B)

# define get right params
padded_params_restricted_3D = padded_params_restricted.reshape(12,D,K)
cat_dim_idx = x_train
padded_params_restricted_chosen = padded_params_restricted_3D[:,:,cat_dim_idx]

# multiply basis with appropriate params
#torch.matmul(basis, theta.unsqueeze(-1)).squeeze(-1) 
prediction = (basis * padded_params_restricted_chosen.T).sum(2)

In [99]:
basis.size()

torch.Size([100, 5, 12])

In [100]:
prediction.size()

torch.Size([100, 5])

In [101]:
padded_params_restricted_chosen.size()

torch.Size([12, 5, 100])

In [102]:
padded_params_restricted_chosen.T.size()

torch.Size([100, 5, 12])

In [103]:
padded_params_restricted_chosen.T[0,0,:]

tensor([-10.0000,  -8.1818,  -6.3636,  -4.5455,  -2.7273,  -0.9091,   0.9091,
          2.7273,   4.5455,   6.3636,   8.1818,  10.0000],
       grad_fn=<SliceBackward0>)

In [104]:
model = GTM(
    number_variables = D,
    number_transformation_layers = 1,
    number_decorrelation_layers = 0,
    degree_transformations = 10,
    spline_transformation = "bernstein",
    transformation_spline_range = (-10, 10),
    device = "cpu") 

/Users/matthiasherp/Desktop/phd_github_repositories/gtm/gtm/gtm_layers/transformation_layer.py:77: UserWarning: Warning: Varying Spline Degree for each Dimension is not implemented for Bernstein, only for B-Spline.
  warnings.warn(
/Users/matthiasherp/Desktop/phd_github_repositories/gtm/gtm/gtm_layers/transformation_layer.py:86: UserWarning: Bernstein polynomial penalization is not implemented yet. only returns zeros hardcoded in bernstein_prediction.py fct
  warnings.warn(


In [105]:
return_dict = model.transformation(input=y_train)

In [106]:
(return_dict["output"] - prediction).abs().max()

tensor(4.7684e-07, grad_fn=<MaxBackward1>)